In [83]:
import pandas as pd
import re

In [84]:
df = pd.read_csv('all_opinions.csv')

# Preprocessing

1) Drop all Nan row

In [85]:
#drop all Nan
df.dropna(inplace= True)
df.sample()

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
31267,Justice Fuller,majority,False,Lambert v. Barrett,1895-11-25,159 U.S. 660,https://www.courtlistener.com/opinion/94293/la...,https://www.courtlistener.com/api/rest/v3/clus...,1895,1895-039,1.0,8.0,0.0,"By section 766 of the Revised Statutes, where ..."


In [86]:
samples=df.copy(deep=True)
sample=samples[:5000]
sample.sample()

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
5131,Justice Fuller,majority,False,Tyler v. Cass County,1892-01-04,142 U.S. 288,https://www.courtlistener.com/opinion/93216/ty...,https://www.courtlistener.com/api/rest/v3/clus...,1892,1891-065,1.0,9.0,0.0,This case comes before us on motion to dismiss...


2) Removing stopwords and splitting into corpus

In [87]:
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()

from nltk.corpus import stopwords
stop_list = stopwords.words('english')
stop_list += ['court', 'law', 'claim', 'state', 'made', 'case', 'act', 'city', 'right', 'rights', 'defendant', 'may', 
              'would', 'one', 'plaintiff', 'upon', 'upon', 'said', 'shall', 'time', 'property', 'could', 'question', 'part', 'fact', 'company', 'held', 'statute', 'united', 'also', 'contract', 'order', 'land', 'power', 'must', 'judgment', '\x97', 'party', 'make', 'without', 'whether', 'subject', 'might', 'purpose', 'provision', 'v.', 'congress', 'decision', 'court,', 'first', 'action', 'interest', 'evidence', 'within', 'two', 'tax', 'authority', 'case,', 'effect', 'person', 'it,', 'government', 'opinion', 'given', 'general', 'use', 'certain', 'cannot', 'that,', 'respect', 'required', 'amount', 'present', 'jurisdiction', 'thus', 'suit', 'proceeding', 'either', 'matter', 'found', 'rule', 'section', 'sale', 'term', 'state,', 'duty', 'title', 'district', 'ground', 'payment', '"the', 'well', 'reason', 'commission', 'it.', 'united_states,', 'decree', 'charge', 'view', 'and,', 'grant', 'corporation', 'paid', 'new', 'taken', 'application', 'point', 'necessary', 'bank', 'another', 'show', 'condition', 'provided', 'used', 'give', 'unless']

In [88]:
sample['text']= sample['text'].apply(lambda x: [word.lower() for word in x.split() if word.lower() not in stop_list])

C:\Users\Yujin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Lemmatization

In [89]:
sample['text']= sample['text'].apply(lambda x: [lemmatizer.lemmatize(word.lower()) for word in x if lemmatizer.lemmatize(word.lower()) not in stop_list])

C:\Users\Yujin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [90]:
sample.sample()

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
6169,Justice Brewer,majority,False,Sawyer v. Piper,1903-04-27,189 U.S. 154,https://www.courtlistener.com/opinion/95839/sa...,https://www.courtlistener.com/api/rest/v3/clus...,1903,1902-111,1.0,9.0,0.0,"[leave, file, supplementary, answer, error, av..."


In [91]:
# sample['sent_list'] = sample['sent_list'].apply(lambda x: [gensim.models.phrases.Phrases(x, min_count=3, threshold=10)])
import gensim 

doc = sample['text'].values.tolist()
bi = gensim.models.phrases.Phrases(doc, min_count=3, threshold=10)
tri = gensim.models.phrases.Phrases(bi[doc], min_count=3, threshold=10)
sample['text'] = tri[bi[doc]]

C:\Users\Yujin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [92]:
df_lem=sample.copy()

In [93]:
df_lem.sample()

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
1787,Justice Reed,majority,False,United States v. Petty Motor Co.,1946-03-25,327 U.S. 372,https://www.courtlistener.com/opinion/104248/u...,https://www.courtlistener.com/api/rest/v3/clus...,1946,1945-048,2.0,7.0,0.0,"[writ_certiorari, judicial_code, §_240, brings..."


# Find common words

In [80]:
from collections import Counter
import numpy as np

doc_lem = df_lem['text'].values.tolist()

a = []

for i in doc_lem:
    a += i


# Pass the split_it list to instance of Counter class. 
Counter = Counter(a) 
  
# most_common() produces k frequently encountered 
# input values and their respective counts. 
most_occur = Counter.most_common(100) 

def Convert(tup, di): 
    for a, b in tup: 
        di.setdefault(a, []).append(b) 
    return di 
dictionary = {}
asd = Convert(most_occur, dictionary)
print(asd.keys())

dict_keys(['upon', 'said', 'shall', 'time', 'property', 'could', 'question', 'part', 'fact', 'company', 'held', 'statute', 'united', 'also', 'contract', 'order', 'land', 'power', 'must', 'judgment', '\x97', 'party', 'make', 'without', 'whether', 'subject', 'might', 'purpose', 'provision', 'v.', 'congress', 'decision', 'court,', 'first', 'action', 'interest', 'evidence', 'within', 'two', 'tax', 'authority', 'case,', 'effect', 'person', 'it,', 'government', 'opinion', 'given', 'general', 'use', 'certain', 'cannot', 'that,', 'respect', 'required', 'amount', 'present', 'jurisdiction', 'thus', 'suit', 'proceeding', 'either', 'matter', 'found', 'rule', 'section', 'sale', 'term', 'state,', 'duty', 'title', 'district', 'ground', 'payment', '"the', 'well', 'reason', 'commission', 'it.', 'united_states,', 'decree', 'charge', 'view', 'and,', 'grant', 'corporation', 'paid', 'new', 'taken', 'application', 'point', 'necessary', 'bank', 'another', 'show', 'condition', 'provided', 'used', 'give', 'unl

# 4.1) Create dictionary

In [94]:
import gensim 
from gensim import corpora

<b> 4.1.1) Create list from dataframe <b>

In [95]:
#df2 lemmatise only
doc_lem= df_lem['text'].values.tolist()

<b> 4.1.2) Create dictionary <b>

In [96]:
dict_lem=corpora.Dictionary(doc_lem)

In [97]:
token_to_id2=dict_lem.token2id

<b> 4.1.3) TF <b>

In [98]:
vec_lem= [dict_lem.doc2bow(doc) for doc in doc_lem]

# Train the LDA model with Gensim

In [99]:
import gensim

<b>1) LDA with term freq (tf)<b>

In [100]:
lda_reviews_lem= gensim.models.ldamodel.LdaModel(corpus=vec_lem, id2word=dict_lem, num_topics=12)

In [101]:
topics_lem=lda_reviews_lem.show_topics(12,15)

for i in range(0,10):
    print(topics_lem[i])

(0, '0.001*"brought" + 0.001*"record" + 0.001*"states," + 0.001*"liability" + 0.001*"public" + 0.001*"account" + 0.001*"see" + 0.001*"think" + 0.001*"creditor" + 0.001*"result" + 0.001*"considered" + 0.001*"appeal" + 0.001*"owner" + 0.001*"policy" + 0.001*"petitioner"')
(1, '0.003*"states," + 0.002*"regulation" + 0.001*"trial" + 0.001*"article" + 0.001*"officer" + 0.001*"word" + 0.001*"good" + 0.001*"although" + 0.001*"every" + 0.001*"therefore" + 0.001*"particular" + 0.001*"policy" + 0.001*"think" + 0.001*"states." + 0.001*"rate"')
(2, '0.002*"company," + 0.002*"business" + 0.001*"public" + 0.001*"agreement" + 0.001*"railroad" + 0.001*"pay" + 0.001*"consideration" + 0.001*"making" + 0.001*"stated" + 0.001*"property," + 0.001*"whole" + 0.001*"lease" + 0.001*"construction" + 0.001*"take" + 0.001*"estate"')
(3, '0.003*"patent" + 0.002*"circuit" + 0.002*"bond" + 0.002*"cause" + 0.001*"brought" + 0.001*"is," + 0.001*"them," + 0.001*"record" + 0.001*"possession" + 0.001*"states," + 0.001*"b

# Choosing Optimal Number of the Topics

<b>1) Model with tf vectors - DONT RUN THIS<b>

In [21]:
# Can take a long time to run. In this case we are going to  k_max=10.
import datetime
print(datetime.datetime.now())

from gensim.models import CoherenceModel

model_list = []
coherence_values = []
model_topics = []

for num_topics in range(2, 14, 2):
    #sg_lda_x = gensim.models.ldamodel.LdaModel(corpus=sg_vecs, id2word=sg_dictionary, num_topics=num_topics)
    lda_reviews_lem= gensim.models.ldamodel.LdaModel(corpus=vec_lem, id2word=dict_lem, num_topics=num_topics, iterations=1000, random_state=100)
    coherencemodel = CoherenceModel(model=lda_reviews_lem, texts=doc_lem, dictionary=dict_lem, coherence='c_v')
    model_topics.append(num_topics)
    model_list.append(lda_reviews_lem)
    coherence_values.append(coherencemodel.get_coherence())
    print("#Topics: " + str(num_topics) + " Score: " + str(coherencemodel.get_coherence()))

print(datetime.datetime.now())

2020-09-26 16:41:00.401077
#Topics: 2 Score: 0.2638833291549765
#Topics: 4 Score: 0.2667767666483527
#Topics: 6 Score: 0.2663717329493514
#Topics: 8 Score: 0.2656337780085692
#Topics: 10 Score: 0.27136799272610207
#Topics: 12 Score: 0.2812567583644125
2020-09-26 17:02:50.345578


In [22]:
# Show graph

import matplotlib.pyplot as plt

limit=14; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

<Figure size 640x480 with 1 Axes>

In [23]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.2639
Num Topics = 4  has Coherence Value of 0.2668
Num Topics = 6  has Coherence Value of 0.2664
Num Topics = 8  has Coherence Value of 0.2656
Num Topics = 10  has Coherence Value of 0.2714
Num Topics = 12  has Coherence Value of 0.2813


# <b>- END HERE BECAUSE RESULT ARE NOT GOOD-<b>

# Coherence Score - dont think need this

In [25]:
from gensim.models import CoherenceModel

In [26]:
# Compute Coherence score for lemmatising -tf only
coherence_model_lda_lem = CoherenceModel(model=lda_reviews_lem, texts=doc_lem, dictionary=dict_lem, coherence='c_v')
coherence_lda_lem = coherence_model_lda_lem.get_coherence()
print('\nCoherence Score LDA-lem: ', coherence_lda_lem)



Coherence Score LDA-lem:  0.2812567583644125


In [102]:
from pprint import pprint
optimal_model = lda_reviews_lem
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.001*"brought" + 0.001*"record" + 0.001*"states," + 0.001*"liability" + '
  '0.001*"public" + 0.001*"account" + 0.001*"see" + 0.001*"think" + '
  '0.001*"creditor" + 0.001*"result"'),
 (1,
  '0.003*"states," + 0.002*"regulation" + 0.001*"trial" + 0.001*"article" + '
  '0.001*"officer" + 0.001*"word" + 0.001*"good" + 0.001*"although" + '
  '0.001*"every" + 0.001*"therefore"'),
 (2,
  '0.002*"company," + 0.002*"business" + 0.001*"public" + 0.001*"agreement" + '
  '0.001*"railroad" + 0.001*"pay" + 0.001*"consideration" + 0.001*"making" + '
  '0.001*"stated" + 0.001*"property,"'),
 (3,
  '0.003*"patent" + 0.002*"circuit" + 0.002*"bond" + 0.002*"cause" + '
  '0.001*"brought" + 0.001*"is," + 0.001*"them," + 0.001*"record" + '
  '0.001*"possession" + 0.001*"states,"'),
 (4,
  '0.003*"water" + 0.003*"river" + 0.002*"public" + 0.002*"states," + '
  '0.001*"vessel" + 0.001*"line" + 0.001*"owner" + 0.001*"navigation" + '
  '0.001*"place" + 0.001*"project"'),
 (5,
  '0.003*"appeal" + 0.00

# Most Dominant Topic for each Document

In [28]:
#Find most dominant topic

import numpy as np
import pandas as pd

def format_topics_sentences(ldamodel, corpus, data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(data)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

#I choose model_list[1] where the number of topics is 4

df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=vec_lem, data=doc_lem)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(100)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,10.0,0.9933,"court, upon, case, made, would, said, may, one...","[case, brought, appeal_decree_circuit, court, ..."
1,1,7.0,0.2333,"said, upon, court, land, made, company, case, ...","[plaintiff_error,_plaintiff_below,, commenced_..."
2,2,5.0,0.3044,"court, case, upon, may, state, defendant, one,...","[whether, state, court, circuit_court_united_s..."
3,3,7.0,0.8122,"said, upon, court, land, made, company, case, ...","[writ_error_circuit_court, united_state_distri..."
4,4,10.0,0.9971,"court, upon, case, made, would, said, may, one...","[case, present, single, point, consideration.,..."
...,...,...,...,...,...
95,95,10.0,0.9853,"court, upon, case, made, would, said, may, one...","[seen, fact, case, plaintiff, default, account..."
96,96,7.0,0.6452,"said, upon, court, land, made, company, case, ...","[suit_foreclose, railroad, mortgage, incidenta..."
97,97,10.0,0.7021,"court, upon, case, made, would, said, may, one...","[stating, case,, proceeded:, 32, several_objec..."
98,98,7.0,0.5099,"said, upon, court, land, made, company, case, ...","[suit_brought, united_state, supreme_court_dis..."


# Save Model

In [ ]:
#Save a model to disk, or reload a pre-trained model
import pickle

# lda_lem_tfidf.save("lda")

ldapickle = open('ldapickle', "wb")
pickle.dump(optimal_model, ldapickle)
ldapickle.close()

In [ ]:
import pickle
reader = open("ldamallet_model478927.pickle", "rb")
tester_model = pickle.load(reader)
topics_lem=tester_model.show_topics(10,15)
topics_lem

In [ ]:
# TEST NEW DATA
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim import corpora
import pickle

lemmatizer= WordNetLemmatizer()

stop_list = stopwords.words('english')
stop_list += ['hotel', 'however', 'could', 'get', 'back', 'bit', 'one', 'know', 'i', 'have', 'would', 'take', 'a', 'choose', 'the', 'first', 'second', 'lovely', 'will', 'definitely', 'longer', 'stayed', 'also']

def preprocessing(review):
    sentences = review.split(". ")
    data = [[word.lower() for word in x.split() if word.lower() not in stop_list] for x in sentences]
#     lem = [[lemmatizer.lemmatize(w) for w in doc] for doc in data]
    dict_lem=corpora.Dictionary(data)
    token_to_id2=dict_lem.token2id
    vec_lem= [dict_lem.doc2bow(doc) for doc in data]
    
    return vec_lem

unseen_rev= preprocessing("room and bed is spacious. shower old put room water. staff friendly reception room helpful bar.")

for sen in unseen_rev:
#     tester_model is the lda model that you load with pickle
    result=tester_model[sen]
    result = sorted(result, key=lambda x: x[1], reverse=True)
    topic = result[0][0]
    print(topic)

In [ ]:
import pickle
import nltk

classifier_saved = open("ldamallet_model478927.pickle", "rb") #binary read
tester_model = pickle.load(classifier_saved)
classifier_saved.close()

# TEST NEW DATA
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim import corpora
import pickle

lemmatizer= WordNetLemmatizer()

stop_list = stopwords.words('english')
stop_list += ['hotel', 'however', 'could', 'get', 'back', 'bit', 'one', 'know', 'i', 'have', 'would', 'take', 'a', 'choose', 'the', 'first', 'second', 'lovely', 'will', 'definitely', 'longer', 'stayed', 'also']

def preprocessing(review):
    sentences = review.split(". ")
    data = [[word.lower() for word in x.split() if word.lower() not in stop_list] for x in sentences]
#     lem = [[lemmatizer.lemmatize(w) for w in doc] for doc in data]
    dict_lem=corpora.Dictionary(data)
    token_to_id2=dict_lem.token2id
    vec_lem= [dict_lem.doc2bow(doc) for doc in data]
    
    return vec_lem

unseen_rev= preprocessing("room and bed is spacious. shower old put room water. staff friendly reception room helpful bar.")

gensim_lda = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(tester_model)

# print(unseen_rev)
for sen in unseen_rev:
#     tester_model is the lda model that you load with pickle
    result=gensim_lda[sen]
    result = sorted(result, key=lambda x: x[1], reverse=True)
    topic = result[0][0]
    print(topic)